In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mp_vis
from lib import aero
from mp import MeteoParticleModel
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# center of the map, appox
mp = MeteoParticleModel(lat0=48, lon0=15, tstep=10)

# Change default MP model parameters
mp.AREA_XY = (-2000, 2000)
mp.GRID_BOND_XY = 100
mp.N_AC_PTCS = 200
mp.AGING_SIGMA = 300
mp.PTC_WALK_XY_SIGMA = 20
mp.ACCEPT_PROB_FACTOR = 1


In [ ]:
# read wind raw measurements
wind = pd.read_csv('data/wind_eu_demo_20min.csv.gz')
print(wind['latitude'].min(), wind['latitude'].max())
print(wind['longitude'].min(), wind['longitude'].max())

wind['latitude'] = wind['latitude'].round(2)
wind['longitude'] = wind['longitude'].round(2)
wind['altitude'] = wind['altitude'].round(-2).astype(int)
wind = wind.groupby(['timestamp', 'latitude', 'longitude', 'altitude']).median().reset_index()
wind = wind.dropna()

ts0 = int(wind.timestamp.min())
ts1 = int(wind.timestamp.max())

In [ ]:
# rum Meteo-Particle model for 10 minutes (a bit slow, go enjoy a cup of coffee)

for t in sorted(wind.timestamp.unique()):
    d = wind[wind.timestamp==t]
    obs = {
        'lat': d.latitude.values,
        'lon': d.longitude.values,
        'alt': d.altitude.values,
        'wx': d.wind_u.values,
        'wy': d.wind_v.values,
        'temp': [0] * d.shape[0]
    }

    mp.sample(obs, acceptprob=True)

    print("time: %d | n_ptc: %d"  % (t, len(mp.PTC_X)))

    if t - ts0 >= 600:  # 10 minute
        break


In [ ]:
# construct example grid
lat = np.arange(32, 64, 0.5)
lon = np.arange(-24, 54, 1)

lats, lons = np.meshgrid(lat, lon)
lats = lats.flatten()
lons = lons.flatten()
alts = len(lats) * [30000]  # 30000ft

data = mp.construct(coords=[lats, lons, alts], xyz=False, confidence=False)

df = pd.DataFrame({
    'u_wind': data[3],
    'v_wind': data[4],
    'lat_': lats,
    'lon_': lons,
}).set_index(['lat_', 'lon_'])

dataxr = df.to_xarray()

In [ ]:
# visualize

from ipyleaflet import Map, Velocity, basemaps

map_ = Map(
    center=(52, 15),
    zoom=4,
    interpolation="linear",
    basemap=basemaps.CartoDB.DarkMatter,
)

wind = Velocity(
    data=dataxr,
    zonal_speed="u_wind",
    meridional_speed="v_wind",
    latitude_dimension="lat_",
    longitude_dimension="lon_",
    velocity_scale=0.002,
    max_velocity=120,
)

map_.add_layer(wind)

map_